<a href="https://www.kaggle.com/code/faridjamalov/https-www-kaggle-com-code-faridjamalov-cnn-br?scriptVersionId=263833300" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import os
import tensorflow as tf

# ------------------------------
# Step 0: Suppress unnecessary TF warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Hides INFO and WARNING logs
# ------------------------------

# ------------------------------
# Step 1: Inspect Kaggle input directory
data_dir = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset"

print("Classes:", os.listdir(data_dir))
for folder in os.listdir(data_dir):
    folder_path = os.path.join(data_dir, folder)
    if os.path.isdir(folder_path):
        print(folder, ":", len(os.listdir(folder_path)))
# ------------------------------

# ------------------------------
# Step 2: Check GPU availability
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth to avoid pre-allocating full GPU memory
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU(s) detected: {gpus}")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU detected. Using CPU.")

# Optional: confirm TensorFlow version and available devices
print("TensorFlow version:", tf.__version__)
print("Available devices:", tf.config.list_physical_devices())
# ------------------------------



What this code does
Suppress unnecessary warnings

Prevents TensorFlow from printing lots of INFO and WARNING messages so the output is cleaner.

Inspect the dataset

Looks into the Kaggle input folder.

Identifies the classes (e.g., tumor vs. no tumor) by checking folder names.

Counts how many images are available in each class to understand dataset balance.

Check GPU availability

Determines whether a GPU is available for faster training.

Sets GPU memory to grow dynamically instead of reserving all memory upfront.

Falls back to CPU if no GPU is found.

Confirm TensorFlow version

Helps ensure your code runs with the correct library version and is compatible with the system setup.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator with augmentation and normalization
datagen = ImageDataGenerator(
    rescale=1./255,           # Normalize pixel values
    shear_range=0.2,          # Random shear
    zoom_range=0.2,           # Random zoom
    horizontal_flip=True,     # Random horizontal flip
    validation_split=0.2      # 20% for validation
)

# Training generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),   # Resize images
    batch_size=32,
    class_mode='binary',      # Binary classification
    subset='training',
    shuffle=True
)

# Validation generator
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=False
)


Summary of what this does:

Normalizes images to the 0–1 range.

Applies data augmentation (shear, zoom, horizontal flip) to make the model more robust.

Splits the dataset into training (80%) and validation (20%).

Prepares iterators (generators) that feed images to the model in batches during training.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import cv2

# -----------------------------
# 1️⃣ Data Generators (senin kodundan)
data_dir = "/kaggle/input/brain-mri-images-for-brain-tumor-detection/brain_tumor_dataset"

datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training',
    shuffle=True
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=False
)

# -----------------------------
# 2️⃣ CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
    MaxPooling2D(2,2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

# -----------------------------
# 3️⃣ Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15
)

# -----------------------------
# 4️⃣ Plot Accuracy & Loss
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend()
plt.title('Accuracy')

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('Loss')
plt.show()

# -----------------------------
# 5️⃣ Confusion Matrix & Classification Report
val_steps = validation_generator.samples // validation_generator.batch_size + 1
preds = model.predict(validation_generator, steps=val_steps)
y_pred = (preds > 0.5).astype(int).ravel()
y_true = validation_generator.classes

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=validation_generator.class_indices, yticklabels=validation_generator.class_indices)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

print(classification_report(y_true, y_pred, target_names=validation_generator.class_indices.keys()))

# -----------------------------
# 6️⃣ Grad-CAM Example (1 test image)
def get_gradcam(model, img_array, layer_name='conv2d_2'):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:,0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# Load a test image from validation set
img_path = validation_generator.filepaths[0]
img = tf.keras.utils.load_img(img_path, target_size=(150,150))
img_array = tf.keras.utils.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

heatmap = get_gradcam(model, img_array)

plt.imshow(img)
plt.imshow(heatmap, cmap='jet', alpha=0.5)  # Overlay heatmap
plt.title("Grad-CAM")
plt.axis('off')
plt.show()


This part of the code is preparing the dataset for model training and validation. It normalizes image pixel values and applies data augmentation (shear, zoom, horizontal flip) to improve model generalization. The ImageDataGenerator creates iterators (generators) that automatically load images in batches, resize them to a consistent shape, and split the data into training (80%) and validation (20%), making the workflow efficient for feeding images into a CNN model.

In [ ]:
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.legend(); plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

y_pred = (model.predict(validation_generator) > 0.5).astype(int).ravel()
y_true = validation_generator.classes

cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
print(classification_report(y_true, y_pred))


This code is evaluating the performance of a trained model on the validation dataset.

model.predict(validation_generator) generates predicted probabilities for each image.

(> 0.5).astype(int) converts probabilities to binary class labels.

confusion_matrix shows the counts of true positives, true negatives, false positives, and false negatives, helping visualize where the model makes mistakes.

sns.heatmap plots the confusion matrix for easy interpretation.

classification_report provides detailed metrics like precision, recall, F1-score, and accuracy for each class.
 Usability:
This helps in assessing how well the model performs, identifying biases, and guiding further improvements in training or preprocessing.

In [1]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=validation_generator.class_indices.keys()))


NameError: name 'y_true' is not defined

In [ ]:
img_path = validation_generator.filepaths[0]
# Load image, preprocess, get heatmap, overlay


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf

# -----------------------------
# 1️⃣ Tahminleri al
val_steps = validation_generator.samples // validation_generator.batch_size + 1
y_pred_probs = model.predict(validation_generator, steps=val_steps)
y_pred = (y_pred_probs > 0.5).astype(int).ravel()
y_true = validation_generator.classes

# -----------------------------
# 2️⃣ Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=validation_generator.class_indices, 
            yticklabels=validation_generator.class_indices)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# -----------------------------
# 3️⃣ Classification Report
print("Classification Report:\n")
print(classification_report(y_true, y_pred, target_names=validation_generator.class_indices.keys()))

# -----------------------------
# 4️⃣ Grad-CAM Fonksiyonu
def get_gradcam(model, img_array, layer_name='conv2d_2'):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:,0]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0,1,2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# -----------------------------
# 5️⃣ Örnek Grad-CAM Görselleştirme
img_path = validation_generator.filepaths[0]
img = tf.keras.utils.load_img(img_path, target_size=(150,150))
img_array = tf.keras.utils.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

heatmap = get_gradcam(model, img_array)

plt.imshow(img)
plt.imshow(heatmap, cmap='jet', alpha=0.5)  # Overlay heatmap
plt.title("Grad-CAM")
plt.axis('off')
plt.show()


This code is focused on evaluating and interpreting the model’s predictions on the validation set:

Predictions (y_pred)

The model predicts probabilities for each validation image.

Probabilities are converted to binary class labels (0 or 1) for evaluation.

Confusion Matrix

Visualizes the model’s correct and incorrect predictions.

sns.heatmap makes it easy to see true positives, true negatives, false positives, and false negatives.

Classification Report

Provides precision, recall, F1-score, and support for each class.

Helps assess overall performance and class-specific performance.

Grad-CAM Placeholder

Indicates the next step would be to implement Grad-CAM, a technique to visualize which parts of an image the CNN focuses on when making predictions.

 Usability:
This section is essential for model evaluation and explainability, helping you identify strengths, weaknesses, and interpret how the CNN makes decisions.

In [ ]:
# A simple Grad-CAM function adapted for Keras
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

How it works:

Builds a gradient model that outputs both the last conv layer’s activations and the model’s predictions.

Computes the gradient of the predicted class with respect to the last conv layer.

Averages gradients across width and height to get importance weights for each feature map.

Computes a weighted sum of feature maps to produce the class activation heatmap.

Normalizes the heatmap between 0 and 1 for visualization.

✅ Usability:

Allows interpretation of CNN decisions by highlighting image regions influencing the prediction.

Useful for debugging, explaining misclassifications, and improving trust in model predictions, especially in medical imaging tasks like brain tumor detection.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.2
)

# Load data from a directory
train_generator = datagen.flow_from_directory(
    'path/to/your/dataset/Training',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    'path/to/your/dataset/Training',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Load the test set separately without augmentation
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'path/to/your/dataset/Testing',
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# Get true labels and predicted labels from the test generator
Y_pred = model.predict(test_generator)
y_pred = np.argmax(Y_pred, axis=1)
y_true = test_generator.classes

# Print classification report
print('Classification Report')
target_names = list(test_generator.class_indices.keys())
print(classification_report(y_true, y_pred, target_names=target_names))

# Plot confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()